In [23]:
import alibi_detect
alibi_detect.od.__all__
from pathlib import Path
import numpy as np
import pandas as pd
from alibi_detect.saving import save_detector, load_detector


In [24]:
from sklearn.model_selection import train_test_split
data = pd.read_csv("..\\data\\urls_with_features_selected.csv")
api_data = pd.read_csv("..\\data\\api_urls.csv")
X = data.loc[:, data.columns != 'type']
y = data['type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
import os, sys
from alibi_detect.od import IForest
from pathlib import Path 

detector_name = 'IForest'

od = IForest(threshold=0.5,  # threshold for outlier score
                 n_estimators=100)
    
    # train
od.fit(X_train)
    
    # save the trained outlier detector
save_detector(od, "\\outlier_detector")

In [26]:
od.fit(X_train)

In [27]:
od.infer_threshold(
    X,
    threshold_perc=95
)

In [36]:
preds = od.predict(
   [[2,1,0,17,0,0,0,0,0,0,True,False,9,0,0],
    [2,1,0,22,0,0,0,0,0,0,True,False,14,0,0],
    [254,51,445,2442,564,46,46546,23,6968,34534,False,False,1544,675,675]],
    return_instance_score=True
)

c:\Users\rosav\Desktop\MalURLs\malurls_env\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [37]:
preds

{'data': {'instance_score': array([-0.10139174, -0.10703387,  0.34267297]),
  'feature_score': None,
  'is_outlier': array([0, 0, 1])},
 'meta': {'name': 'IForest',
  'online': False,
  'data_type': 'tabular',
  'version': '0.11.5dev',
  'detector_type': 'outlier'}}

In [ ]:
labels = outlier_batch.target_names
y_pred = od_preds['data']['is_outlier']
f1 = f1_score(y_outlier, y_pred)
print('F1 score: {:.4f}'.format(f1))
cm = confusion_matrix(y_outlier, y_pred)
df_cm = pd.DataFrame(cm, index=labels, columns=labels)
sns.heatmap(df_cm, annot=True, cbar=True, linewidths=.5)
plt.show()